nb

In [1]:
using BenchmarkTools, InteractiveUtils

nb

# Types / struct
As opposed to Object-Oriented-Perogramming, one of the key features of Julia is the so-called multiple dispatch. It is a powerful
code dessign pattern that allows to write (typically highly abstract) generic code.
From a simple point of view, multiple dispatch means that you can write several functions
with the same name, but with different methods depending on the number and types of the arguments

In [2]:
foo(x::Float64) = 2*x
foo(x::Int64) = 4*x
foo(x) = "Hello"

foo (generic function with 3 methods)

In [3]:
foo(1)

4

In [4]:
foo(1.0)

2.0

In [5]:
foo(2f0)

"Hello"

This means that I could extend a given method for a type that is defined in an external package.
Typical one wants to create its own dat type, this is done with `struct`. For example we can create a
`Dual` number type.

In [6]:
struct Dual1
    val
    partial
end

where the Dual number $x + a\epsilon \rightarrow f(x) + f'(x)\epsilon$ is similar to an imaginary, with $\epsilon^2 = 0$.
Dual numbers are used in the so-called forward mode autodifferentiation, and yield machine-precission derivatives.
The easiest implimentation is done via "operator overloading", which means that we expand basic arithemtic operations
to Dual numbers arithmetics. Using multiple dispatch, we can define the rules for addition, multiplication, and exponentiation:

In [7]:
import Base: +, *, ^
(+)(x::Dual1 , y::Dual1)  = Dual1(x.val + y.val, x.partial + y.partial)
(+)(x::Dual1 , y::Number) = Dual1(y, 0.0) + x
(+)(x::Number, y::Dual1)  = Dual1(x, 0.0) + y
(*)(x::Dual1 , y::Dual1)  = Dual1(x.val * y.val, x.val * y.partial + y.val * x.partial)
(*)(x::Dual1 , y::Number) = Dual1(y, 0.0) * x
(*)(x::Number, y::Dual1)  = Dual1(x, 0.0) * y
(^)(x::Dual1 , n::Number) = Dual1(x.val^n, n*x.partial * x.val^(n-1))

^ (generic function with 75 methods)

In [8]:
f(x) = x^2 + 2*x + 1
x1 = Dual1(3.0, 1.0)
f(x1)

Main.var"##432".Dual1(16.0, 8.0)

Ok, it works. Now lets benchmark it, the operations are trivial so we should expect speed and no allocations....

In [9]:
@btime f($x1)

  227.657 ns (11 allocations: 176 bytes)


Main.var"##432".Dual1(16.0, 8.0)

or not. Unexpected allocations are usually due to some type instability, so lets see what `@code_warntype` says

In [10]:
@code_warntype f(x1)

MethodInstance for Main.var"##432".f(::Main.var"##432".Dual1)
  from f(x) @ Main.var"##432" ~/Desktop/Seminars/JuliaMainz23/types.ipynb:1
Arguments
  #self#::Core.Const(Main.var"##432".f)
  x::Main.var"##432".Dual1
Body::Main.var"##432".Dual1
1 ─ %1 = Main.var"##432".:^::Core.Const(^)
│   %2 = Core.apply_type(Base.Val, 2)::Core.Const(Val{2})
│   %3 = (%2)()::Core.Const(Val{2}())
│   %4 = Base.literal_pow(%1, x, %3)::Main.var"##432".Dual1
│   %5 = (2 * x)::Main.var"##432".Dual1
│   %6 = (%4 + %5 + 1)::Main.var"##432".Dual1
└──      return %6



it seems it's not bad, everything is in blue. Lets take a better look using JET.jl

In [11]:
using JET
@report_opt f(x1)

═════ 9 possible errors found ═════
┌ f(x::Main.var"##432".Dual1) @ Main.var"##432" /Users/albert/Desktop/Seminars/JuliaMainz23/types.ipynb:1
│┌ literal_pow(f::typeof(^), x::Main.var"##432".Dual1, ::Val{2}) @ Base ./intfuncs.jl:338
││┌ ^(x::Main.var"##432".Dual1, n::Int64) @ Main.var"##432" /Users/albert/Desktop/Seminars/JuliaMainz23/types.ipynb:8
│││ runtime dispatch detected: (%1::Any Main.var"##432".:^ n::Int64)::Any
││└────────────────────
││┌ ^(x::Main.var"##432".Dual1, n::Int64) @ Main.var"##432" /Users/albert/Desktop/Seminars/JuliaMainz23/types.ipynb:8
│││ runtime dispatch detected: (%4::Any Main.var"##432".:^ %5::Int64)::Any
││└────────────────────
││┌ ^(x::Main.var"##432".Dual1, n::Int64) @ Main.var"##432" /Users/albert/Desktop/Seminars/JuliaMainz23/types.ipynb:8
│││ runtime dispatch detected: Main.var"##432".:*(n::Int64, %3::Any, %6::Any)::Any
││└────────────────────
│┌ *(x::Int64, y::Main.var"##432".Dual1) @ Main.var"##432" /Users/albert/Desktop/Seminars/JuliaMainz23/types.i

it seems that the problem is that the compiler is not able to infer the types of `Dual1.val` and `Dual1.partial`, leading to runtime dispatch.
This is due to the fact that we did not specify the type of the fields of the struct. We can do it in the following way by parameterizing the `struct`

In [12]:
struct Dual2{T}
    val::T
    partial::T
end
(+)(x::Dual2   , y::Dual2)            = Dual2(x.val + y.val, x.partial + y.partial)
(+)(x::Dual2{T}, y::Number)   where T = Dual2(T(y), 0.0) + x
(+)(x::Number  , y::Dual2{T}) where T = Dual2(T(x), 0.0) + y
(*)(x::Dual2   , y::Dual2)            = Dual2(x.val * y.val, x.val * y.partial + y.val * x.partial)
(*)(x::Dual2{T}, y::Number)   where T = Dual2(T(y), 0.0) * x
(*)(x::Number  , y::Dual2{T}) where T = Dual2(T(x), 0.0) * y
(^)(x::Dual2   , n::Number)           = Dual2(x.val^n, n*x.partial * x.val^(n-1))

^ (generic function with 76 methods)

In [13]:
x2 = Dual2(3.0, 1.0)
f(x2)

Main.var"##432".Dual2{Float64}(16.0, 8.0)

In [14]:
@btime f($x2)

  5.333 ns (0 allocations: 0 bytes)


Main.var"##432".Dual2{Float64}(16.0, 8.0)

In [15]:
@code_warntype f(x2)

MethodInstance for Main.var"##432".f(::Main.var"##432".Dual2{Float64})
  from f(x) @ Main.var"##432" ~/Desktop/Seminars/JuliaMainz23/types.ipynb:1
Arguments
  #self#::Core.Const(Main.var"##432".f)
  x::Main.var"##432".Dual2{Float64}
Body::Main.var"##432".Dual2{Float64}
1 ─ %1 = Main.var"##432".:^::Core.Const(^)
│   %2 = Core.apply_type(Base.Val, 2)::Core.Const(Val{2})
│   %3 = (%2)()::Core.Const(Val{2}())
│   %4 = Base.literal_pow(%1, x, %3)::Main.var"##432".Dual2{Float64}
│   %5 = (2 * x)::Main.var"##432".Dual2{Float64}
│   %6 = (%4 + %5 + 1)::Main.var"##432".Dual2{Float64}
└──      return %6



# # Note on structs
- structs are immutable by default, but you can make them mutable by adding `mutable struct`

In [16]:
mutable struct Foo1{T}
    a::T
end
a = Foo1(5)
a.a = 6

6

- Array *values* are mutable, but the array itself is not.

In [17]:
struct Foo2{T}
    a::T
end
b = Foo2(zeros(5))
b.a[1] = 1.0

1.0

- We can also use references to mutate scalar values in immutable structs

In [18]:
struct Bar{T}
    a::Ref{T}
end
c = Bar(Ref(5))
c.a[] = 6

6

In [19]:
@btime $a.a = 6
@btime $c.a[] = 6

  1.333 ns (0 allocations: 0 bytes)
  1.666 ns (0 allocations: 0 bytes)


6

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*